### Nonlinear Problems in Finance (MATHGR5400), Columbia University, Spring 2019
# Homework II

### Due Date: 11:55 PM Tuesday, March 12, 2019
You should turn in the notebook at Columbia CourseWorks website

Please comment your code properly.

Before you turn in the notebook, press the "Run all cells" button in the toolbar, and make sure all the calculation results and graphs are produced correctly in a reasonable time frame, and then save the notebook.

In [1]:
%matplotlib inline
import itertools
import scipy
import math
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import laguerre
from scipy.interpolate import interp1d
from scipy.stats import norm
plt.rc('figure', figsize=(6, 5))
plt.rc('axes', grid=True, xmargin=0, ymargin=0, autolimit_mode='round_numbers')

In [2]:
def blackscholes_mc(S=100, vol=0.2, r=0, q=0, ts=np.linspace(0, 1, 13), npaths=10):
    """Generate Monte-Carlo paths in Black-Scholes model.

    Parameters
    ----------
    S: scalar
        The spot price of the underlying security.
    vol: scalar
        The implied Black-Scholes volatility.
    r: scalar
        The annualized risk-free interest rate, continuously compounded.
    q: scalar
        The annualized continuous dividend yield.
    ts: array_like
        The time steps of the simualtion
    npaths: int
        the number of paths to simulate

    Returns
    -------
    paths: ndarray
        The Monte-Carlo paths.
    """
    nsteps = len(ts) - 1
    ts = np.asfarray(ts)[:, np.newaxis]
    W = np.cumsum(np.vstack((np.zeros((1, npaths), dtype=np.float),
                             np.random.randn(nsteps, npaths) * np.sqrt(np.diff(ts, axis=0)))),
                  axis=0)
    paths = np.exp(-0.5*vol**2*ts + vol*W)*S*np.exp((r-q)*ts)
    return paths


def blackscholes_price(K, T, S, vol, r=0, q=0, callput='call'):
    """Compute the call/put option price in the Black-Scholes model
    
    Parameters
    ----------
    K: scalar or array_like
        The strike of the option.
    T: scalar or array_like
        The maturity of the option, expressed in years (e.g. 0.25 for 3-month and 2 for 2 years)
    S: scalar or array_like
        The current price of the underlying asset.
    vol: scalar or array_like
        The implied Black-Scholes volatility.
    r: scalar or array_like
        The annualized risk-free interest rate, continuously compounded.
    q: scalar or array_like
        The annualized continuous dividend yield.
    callput: str
        Must be either 'call' or 'put'.

    Returns
    -------
    price: scalar or array_like
        The price of the option.

    Examples
    --------
    >>> blackscholes_price(95, 0.25, 100, 0.2, r=0.05, callput='put')
    1.5342604771222823
    """
    F = S*np.exp((r-q)*T)
    v = np.sqrt(vol**2*T)
    d1 = np.log(F/K)/v + 0.5*v
    d2 = d1 - v
    try:
        opttype = {'call':1, 'put':-1}[callput.lower()]
    except:
        raise ValueError('The value of callput must be either "call" or "put".')
    price = opttype*(F*norm.cdf(opttype*d1)-K*norm.cdf(opttype*d2))*np.exp(-r*T)
    return price

# Dual Problem and Upper Bounds for American Options

<strong>Primal Problem</strong> (maximizing over stopping times)

\begin{equation*}
V_t=\sup_{\tau\in\mathcal{T}_{t, T}}\mathbb{E}^\mathbb{Q}\left[D_{t, \tau}F_{\tau}\big|\mathcal{F}_t\right]
\end{equation*}

<strong>Dual Problem</strong> (minimizing over martingales) [Rogers 2002; Haugh and Kogan 2004]

\begin{equation*}
V_t=\inf_{M\in\mathcal{M}_{t, 0}}\mathbb{E}^\mathbb{Q}\left[\sup_{t\leq s\leq T}\left(D_{t, s}F_s-M_s\right)\bigg|\mathcal{F}_t\right].
\end{equation*}

where $\mathcal{M}_{t, 0}$ denotes the set of all right-continuous martingales $(M_s, s\in[t, T])$ with $M_t=0$.
The optimal martingale $M^{\ast}$ is the martingale part of the Doob-Meyer decomposition of $(S_s-S_t)/D_{0, t}$, $t\leq s\leq T$, where $S$ is the Snell envelope of the discounted payoff, i.e., $S_t=\sup_{\tau\in\mathcal{T}_{t,T}}\mathbb{E}^{\mathbb{Q}}\left[D_{0,\tau}F_{\tau}\right]$.

In particular, any martingale $M_s$ with $M_0=0$ gives an upper bound for the price at time 0, $V_0$:

$$V_0 \leq\mathbb{E}^{\mathbb{Q}}\left[\sup_{0\leq s\leq T}\left(D_sF_s-M_s\right)\right].$$

<b style="color:darkorange">Question 1.</b> Consider pricing a one-year Bermudan put option with monthly exercise, where the asset price process is assumed to follow geometric Brownian motion with $S_0=100$, $\sigma=0.2$, $r=0.1$, $q=0.02$, $K=100$ and exercise dates $t_1=\frac{1}{12}$, $t_2=\frac{2}{12}$, $\cdots$, $t_{12}=1$.

Simulate 100,000 paths and use the following martingale to find a upper bound.

(a). $M_t\equiv0$.

(b). $M_t$ is the discounted European put price with the same final maturity less the initial price.

For your reference, the price of the option is $5.152$.

#### Question 1a.

In [3]:
S = 100
vol = 0.2
r = 0.1
q = 0.02
K = 100
T = 1
ts = np.linspace(0, 1, 13)
npaths = 100000

#generate 100000 independent paths and remove time 0 
paths = np.delete(blackscholes_mc(S, vol, r, q, ts=ts, npaths=100000),0, axis = 0)

#calculate discounted payoff at all times and paths, then calculate the max for each path, then take average
np.mean(np.amax(np.exp(-r * (ts[1:] - ts[0]))[:,np.newaxis] * np.maximum(K-paths, 0), axis = 0))

8.61688506743941

#### Question 1b.

In [4]:
S = 100
vol = 0.2
r = 0.1
q = 0.02
K = 100
T = 1
ts = np.linspace(0, 1, 13)
npaths = 100000

#generate 100000 independent paths
paths = blackscholes_mc(S, vol, r, q, ts=ts, npaths=100000)

#evaluate European put prices
arr = np.array([blackscholes_price(K, T - ts[i],paths[i] , vol=0.2, r=0.1, q=0.02, callput='put') for i in range(1,len(paths))])

#calculate payoff less Euro put at all times and paths, then discount to get discounted payoff less the discounted martingale/european price
arr = np.exp(-r * (ts[1:] - ts[0]))[:,np.newaxis] * (np.maximum(K-paths[1:], 0) - arr)

#add initial value back in
arr = (arr + blackscholes_price(K, T, paths[0] , vol=0.2, r=0.1, q=0.02, callput='put'))

#calculate max for each path then take average
np.mean(np.amax(arr, axis = 0))

/home/terminal_1/anaconda3/envs/pyfinance/lib/python3.6/site-packages/ipykernel_launcher.py:65: RuntimeWarning: divide by zero encountered in true_divide


5.348760683621992

## Optimal martingale

The optimal martingale that gives zero <em>duality gap</em> is the martingale component of the discounted true value process (known as Snell envelope) $\frac{V_t}{D_{0,t}}$, following the optimal exercise strategy.

To get a good upper bound, we first find a value process that is close to the true value process, and then extract its martingale component. This approximate value process can be obtained from a functional approximation such as regression, or from an exercise strategy (stopping time).

In general, for a stochastic process $\{U_n\}_{n\geq0}$, to extract its martingale component, we only need to let $M_0=0$, and then

$$M_{n+1}-M_n=U_{n+1}-\mathbb{E}\left[U_{n+1}\big|\mathcal{F}_n\right],\quad n=0,1,\cdots.$$

As a demonstration, we show below the code that calculates the upper bound estimate of the price of Bermudan put with monthly exercises as in Question 1.1. The optimal exercise frontier is given for each exercise date (from a PDE solver), i.e. if the underlying price falls below Bs[i] at time ts[i], then we should exercise the option, otherwise we should continue.

In [5]:
Bs = np.array([87.5297, 87.7804, 88.0589, 88.3694, 88.724, 89.1309, 89.6067, 90.1723, 90.8653, 91.7451, 92.956, 94.8524, 100])

def exer_or_cont(i, S):
    return S <= Bs[i]

In [6]:
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
ts = np.linspace(0, 1, 13)
exer_func = lambda S: np.maximum(K-S, 0)

# nested simulation from time t_i when stock price is S
def nested_mc(S, vol, r, q, i, ts, nnested):
    nested_paths = np.full(nnested, S, dtype=np.float)
    tot_payoff = 0
    for j in range(i+1, len(ts)):
        dt = ts[j] - ts[j-1]
        dW = np.random.randn(len(nested_paths))*np.sqrt(dt)        # Brownian increment
        nested_paths = nested_paths*np.exp((r-q)*dt)*np.exp(-0.5*vol**2*dt + vol*dW)
        exer_vals =  exer_func(nested_paths)
        if j < len(ts)-1:
            ind = exer_or_cont(j, nested_paths)                    # identify the paths that need exercise
            tot_payoff += np.sum(exer_vals[ind])*np.exp(-r*ts[j])
            nested_paths = nested_paths[~ind]                      # remove exercised paths 
            if len(nested_paths) == 0:                             # if exercised for all paths, stop
                break
        else:
            tot_payoff +=  np.sum(exer_vals)*np.exp(-r*ts[j])
    return tot_payoff/nnested                                      # taking average of paths

# Simulate independent paths and exercise the option acoording to this strategy
V0 = nested_mc(S0, vol, r, q, 0, ts, 1000000)

# Upper bound by Andersen-Broadie algorithm
npaths = 500                                                       # number of paths in the second independent run
nnested = 1000                                                     # number of paths used in nested simulation
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
V = np.full(paths.shape, np.nan, dtype=np.float)                   # discounted value process V_i (discounted to time zero)
EV = np.full(paths.shape, np.nan, dtype=np.float)                  # Conditional expectation E[V_{i+1}|F_i], or continuation value
#V[0] = V0                                                          # Initial value from lower bound simulation
EV[0] = V0                                                         # at time 0, option value = continuation value
for i in range(1, len(ts)-1):
    exer_vals =  exer_func(paths[i])          
    ind = exer_or_cont(i, paths[i])                                # True for exercise False for continue
    for j in np.nonzero(ind)[0]:
        V[i, j] = exer_vals[j]*np.exp(-r*ts[i])                    # if exercised, V[i,j] = exercise value
        EV[i,j] = nested_mc(paths[i, j], vol, r, q, i, ts, nnested) # launch nested simulation to estimate E[V_{i+1}|F_i]
    for j in np.nonzero(~ind)[0]:
        V[i,j] = nested_mc(paths[i, j], vol, r, q, i, ts, nnested)  # if continue, use nested simulation to estimate V[i, j]
        EV[i, j] = V[i, j]                                         # E[V_{i+1}|F_i] = V_i
V[-1] = exer_func(paths[-1])*np.exp(-r*ts[-1])                     # values at final maturity
hedges = np.zeros(paths.shape, dtype=np.float)
hedges[1:] = np.cumsum(V[1:]-EV[:-1], axis=0)                      # martinglae increment V_{i+1}-E[V_{i+1}|F_i]

print('Primal Price (Lower Bound) = {:.4f}'.format(V0))
print('Dual Price   (Upper Bound) = {:.4f}'.format(np.mean(np.amax(exer_func(paths[1:])*np.exp(-r*ts[1:, np.newaxis])-hedges[1:], axis=0))))

Primal Price (Lower Bound) = 5.1491
Dual Price   (Upper Bound) = 5.1599


Since the strategy is optimal, we see that both lower bound and upper bound prices converge to the true value. 

However, in general, we may only get a sub-optimal strategy (e.g. from Longstaff-Schwartz algorithm). But with this sub-optimal strategy, we can build the (discounted) value process and then extract its martingale component to obtain an upper bound. Intuitively speaking, the closer this sub-optimal strategy is to the optimal one, the tighter the corresponding upper bound.

<b style="color:darkorange">Question 2.</b> Consider pricing the Bermudan put option as in Question 1.1 by using the constant $1.0$ and the Black-Scholes price of a European put option with volatility $0.2$ and maturity $T-t_n$ as the two basis functions at time $t_n$. Use the Longstaff-Schwartz algorithm to build an exercise strategy with these basis functions and then compute the corresponding upper bound.

For your convenience, an implementation of Longstaff-Schwartz algorithm is included in the cell below. It computes the regression coefficients at each exercise date.

In [7]:
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
ts = np.linspace(0, 1, 13)

npaths = 10000
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
betas_LS = np.full((len(ts), 2), np.nan, dtype=np.float)
payoff = np.maximum(K-paths[-1], 0)
for i in range(len(ts)-2, 0, -1):
    discount = np.exp(-r*(ts[i+1]-ts[i]))
    payoff = payoff*discount
    Z = blackscholes_price(K, ts[-1]-ts[i], paths[i], vol, r, q, callput='put')
    A = np.vstack((np.ones_like(Z), Z)).T
    betas_LS[i] = np.linalg.lstsq(A, payoff, rcond=None)[0]    # regression to estimate continuation values
    contval = betas_LS[i, 0]+betas_LS[i, 1]*Z
    exerval = np.maximum(K-paths[i], 0)
    # identify the paths where we should exercise
    ind = exerval > contval
    payoff[ind] = exerval[ind]                     # update payoff on exercised paths
#betas_LS                                           # No regression needed at first and last time steps, return NaN

#### Question 2

In [8]:
S = 100
vol = 0.2
r = 0.1
q = 0.02
K = 100
T = 1
ts = np.linspace(0, 1, 13)
nnested = 500 

#modify nested MC function for use with LS algorithm and with basis functions
def nested_mc_LS(S, vol, r, q, i, ts, nnested):
    nested_paths = np.full(nnested, S, dtype=np.float)
    tot_payoff = 0
    for j in range(i+1, len(ts)):
        dt = ts[j] - ts[j-1]
        dW = np.random.randn(len(nested_paths))*np.sqrt(dt)        # Brownian increment
        nested_paths = nested_paths*np.exp((r-q)*dt)*np.exp(-0.5*vol**2*dt + vol*dW)
        exer_vals =  exer_func(nested_paths)
        if j < len(ts)-1:
            cont_vals = 1 * betas_LS[j][0] +  betas_LS[j][1] * blackscholes_price(K, T - ts[j],nested_paths , vol=0.2, r=0.1, q=0.02, callput='put')
            ind = cont_vals <= exer_vals
            tot_payoff += np.sum(exer_vals[ind])*np.exp(-r*ts[j])
            nested_paths = nested_paths[~ind]                      
            if len(nested_paths) == 0:                             
                break
        else:
            tot_payoff +=  np.sum(exer_vals)*np.exp(-r*ts[j])
    return tot_payoff/nnested                                      # taking average of paths

#generate 100000 independent paths
paths = blackscholes_mc(S, vol, r, q, ts=ts, npaths=5000)

#Andersen-Broadie algorithm similar to demonstration                                                    
V = np.full(paths.shape, np.nan, dtype=np.float)                   
EV = np.full(paths.shape, np.nan, dtype=np.float)                                                    

#calculate first conditional expectation                                                     
EV[0] = nested_mc_LS(S0, vol, r, q, 0, ts, 1000000)      

for i in range(1, len(ts)-1):
    exer_vals =  exer_func(paths[i])          
    cont_vals = 1 * betas_LS[i][0] +  betas_LS[i][1] * blackscholes_price(K, T - ts[i],paths[i] , vol=0.2, r=0.1, q=0.02, callput='put')
    ind = cont_vals <= exer_vals
    
    # True for exercise False for continue
    for j in np.nonzero(ind)[0]:
        V[i, j] = exer_vals[j]*np.exp(-r*ts[i])                    
        EV[i,j] = nested_mc_LS(paths[i, j], vol, r, q, i, ts, nnested) 
    for j in np.nonzero(~ind)[0]:
        V[i,j] = nested_mc_LS(paths[i, j], vol, r, q, i, ts, nnested)  
        EV[i, j] = V[i, j]   

#calculate final value of estimated snell envelope
V[-1] = exer_func(paths[-1])*np.exp(-r*ts[-1])       

#calculate the martingale
hedges = np.zeros(paths.shape, dtype=np.float)
hedges[1:] = np.cumsum(V[1:]-EV[:-1], axis=0)               

#calculate discounted payoff less the martingale, find the max on each path, then average
np.mean(np.amax(((exer_func(paths[1:])*np.exp(-r*ts[1:, np.newaxis]))-hedges[1:]), axis=0))

5.17050081558056

<b style="color:darkorange">Question 3.</b> <b>(An alternative algorithm to estimate the upper bound)</b> The primal algorithm (Longstaff-Schwartz or Tsitsiklis-Van Roy) provides a function approximation (via regression) of the continuation value $C_n$ at $t_n$, hence of the value function $V_n = \max(C_n,F_n)$ where $F_n$ is the exercise value. We can use this functional approximation of the value function to write an alternative algorithm in order to estimate the upper bound: at each time $t_n$, use the functional approximation of $V_n$, and run the nested simulation to estimate the conditional expectation $\mathbb{E}\left[V_{n+1}\big|\mathcal{F}_n\right]$ (using the functional approximation of $V_{n+1}$). Note that this nested simulation only needs to be run for one time step.

For the same Bermudan put option and same basis functions, use TVR algorithm to build the value process and then extract its martingale component using the alternative algorithm explained above to estimate the upper bound price.

For your convenience, an implementation of TVR algorithm is included in the cell below. It computes the regression coefficients at each exercise date. 

In [9]:
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
ts = np.linspace(0, 1, 13)

npaths = 100000
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
betas_TVR = np.full((len(ts), 2), np.nan, dtype=np.float)
V2 = np.maximum(K-paths[-1], 0)
for i in range(len(ts)-2, 0, -1):
    discount = np.exp(-r*(ts[i+1]-ts[i]))
    Z = blackscholes_price(K, ts[-1]-ts[i], paths[i], vol, r, q, callput='put')
    A = np.vstack((np.ones_like(Z), Z)).T
    betas_TVR[i] = np.linalg.lstsq(A, V2*discount, rcond=None)[0]      # regression to estimate continuation values
    contval = betas_TVR[i, 0]+betas_TVR[i, 1]*Z
    exerval = np.maximum(K-paths[i], 0)
    V2 = np.maximum(exerval, contval)                      # compute values
#betas_TVR                                                 # No regression needed at first and last time steps, return NaN

#### Question 3

In [10]:
S = 100
vol = 0.2
r = 0.1
q = 0.02
K = 100
T = 1
ts = np.linspace(0, 1, 13)
nnested = 500

#modify nested MC function for TVR and for running just one time step
def nested_mc_TVR_1ts(S, vol, r, q, i, ts, nnested):
    nested_paths = np.full(nnested, S, dtype=np.float)
    dt = ts[i+1] - ts[i]
    dW = np.random.randn(len(nested_paths))*np.sqrt(dt)   
    nested_paths = nested_paths*np.exp((r-q)*dt)*np.exp(-0.5*vol**2*dt + vol*dW)
    exer_vals =  exer_func(nested_paths)
    if i == len(ts) - 2:
        return np.mean(exer_vals)  * np.exp(-r * ts[i+1])
    cont_vals = 1 * betas_TVR[i+1][0] +  betas_TVR[i+1][1] * blackscholes_price(K, T - ts[i+1],nested_paths , vol=0.2, r=0.1, q=0.02, callput='put')
    return np.mean(np.maximum(exer_vals, cont_vals)) * np.exp(-r * ts[i+1])

#generate 100000 independent paths
paths = blackscholes_mc(S, vol, r, q, ts=ts, npaths=10000)

#Andersen-Broadie algorithm similar to demonstration                                                    
V = np.full(paths.shape, np.nan, dtype=np.float)                   
EV = np.full(paths.shape, np.nan, dtype=np.float)                                                    

#calculate first conditional expectation                                                     
EV[0] = nested_mc_TVR_1ts(S0, vol, r, q, 0, ts, 1000000)      

for i in range(1, len(ts)-1):
    exer_vals =  exer_func(paths[i])          
    cont_vals = (1 * betas_TVR[i][0]) +  (betas_TVR[i][1] * blackscholes_price(K, T - ts[i],paths[i] , vol=0.2, r=0.1, q=0.02, callput='put'))
    V[i] = np.maximum(cont_vals, exer_vals) * np.exp(-r * ts[i])
    for j in range(0, len(paths[i])):
        EV[i,j] = nested_mc_TVR_1ts(paths[i,j], vol, r, q, i, ts, nnested) 

#calculate final value of estimated snell envelope
V[-1] = exer_func(paths[-1])*np.exp(-r*ts[-1])       

#calculate the martingale
hedges = np.zeros(paths.shape, dtype=np.float)
hedges[1:] = np.cumsum(V[1:]-EV[:-1], axis=0)               

#calculate discounted payoff less the martingale, find the max on each path, then average
np.mean(np.amax(((exer_func(paths[1:])*np.exp(-r*ts[1:, np.newaxis]))-hedges[1:]), axis=0))

5.172151158933239

The upper bound estimator can be used to check if the basis functions used in regression are adequate. A tight duality gap between the lower bound and upper bound is a confirmation that the linear span of the selected basis functions indeed gives a good approximation to the continuation value. We now illustrate this with the pricing of Bermudan-Asian call options.

<b style="color:darkorange">Question 4</b> (<b>Bermudan-Asian Call Option</b>). In Question 2.5 of Homework I, we priced a Bermudan-Asian call option using a particular set of basis functions. Assume that all the parameters stay the same.

<b>(a).</b> Reuse the exercise strategy derived from the basis functions used in Question 2.5 of Homework I by Longstaff-Schwartz with $\bar{\sigma}=0.1$ to estimate the lower and upper bound. For your convenience, an implementation of Longstaff-Schwartz is included in the cell below. Note that the lower bound should be obtained by running new independent paths stopped according to the strategy from the Longstaff-Schwartz algorithm.

<b>(b).</b> As an alternative set of basis functions at time $t_n$, use 
$$1, A_{t_n}, S_{t_n}, A_{t_n}^2, S_{t_n}^2, A_{t_n}S_{t_n}.$$
in the Longstaff-Schwartz algorithm and then give the corresponding lower bound and upper bound.

<b>(c).</b> Compare the numerical duality gaps obtained from (a) and (b). Which set of basis functions does a better job at estimating the optimal exercise strategy?

In [11]:
# Longstaff-Schwartz
S0, K, vol, r, q = 100, 100, 0.2, 0, 0
ts = np.linspace(0, 1, 13)
npaths = 100000
pathsS = blackscholes_mc(S0, vol, r, q, ts=ts, npaths=npaths)                                  # stock prices
pathsA = np.vstack((pathsS[0], np.cumsum(pathsS[1:], axis=0)/np.arange(1, 13)[:, np.newaxis])) # running averages
vol_ = 0.1
payoff = np.maximum(pathsA[-1]-K, 0)
betas_AC = np.full((len(ts), 2), np.nan, dtype=np.float)
for i in range(len(ts)-2, 0, -1):
    discount = np.exp(-r*(ts[i+1]-ts[i]))
    payoff = payoff*discount
    Z = (i*pathsA[i]+(12-i)*pathsS[i])/12
    callZ = blackscholes_price(K, ts[-1]-ts[i], Z, vol_, callput='call')
    A = np.vstack((np.ones_like(Z), callZ)).T
    betas_AC[i] = np.linalg.lstsq(A, payoff, rcond=None)[0]           # regression to estimate continuation values
    contval = betas_AC[i, 0]+betas_AC[i, 1]*callZ
    exerval = np.maximum(pathsA[i]-K, 0)
    ind = exerval > contval                               # identify the paths where we should exercise
    payoff[ind] = exerval[ind]
#betas_AC

#### Question 4a.)

In [12]:
S0, K, vol, r, q, vol_ = 100, 100, 0.2, 0, 0, 0.1
ts = np.linspace(0, 1, 13)
nnested = 500 

#modify nested MC function for using running average stock price in the payoff rather than the current stock price
def nested_mc_AC(S, vol, r, q, i, ts, nnested, Av):
    nested_paths = np.full(nnested, S, dtype=np.float)
    nested_Av = np.full(nnested, Av, dtype=np.float)
    tot_payoff = 0
    for j in range(i+1, len(ts)):
        dt = ts[j] - ts[j-1]
        dW = np.random.randn(len(nested_paths))*np.sqrt(dt)        # Brownian increment
        nested_paths = nested_paths*np.exp((r-q)*dt)*np.exp(-0.5*vol**2*dt + vol*dW)
        
        nested_Av = (nested_paths + (nested_Av * (j-1))) / (j)
        
        exer_vals =  np.maximum(nested_Av - K, 0)
        if j < len(ts)-1:
            
            Z = (j*nested_Av+(12-j)*nested_paths)/12
            callZ = blackscholes_price(K, ts[-1]-ts[j], Z, vol_, callput='call')        
            cont_vals = betas_AC[j, 0]+betas_AC[j, 1]*callZ
            ind = cont_vals <= exer_vals
            tot_payoff += np.sum(exer_vals[ind])*np.exp(-r*ts[j])
            nested_paths = nested_paths[~ind]
            nested_Av = nested_Av[~ind]
            if len(nested_paths) == 0:                             
                break
        else:
            tot_payoff +=  np.sum(exer_vals)*np.exp(-r*ts[j])
    return tot_payoff/nnested                                      # taking average of paths

In [13]:
#generate 10000 independent paths
pathsS = blackscholes_mc(S0, vol, r, q, ts=ts, npaths=1000)                             
pathsA = np.vstack((pathsS[0], np.cumsum(pathsS[1:], axis=0)/np.arange(1, 13)[:, np.newaxis]))

#Andersen-Broadie algorithm similar to demonstration                                                    
V = np.full(pathsS.shape, np.nan, dtype=np.float)                   
EV = np.full(pathsS.shape, np.nan, dtype=np.float)                                                    

#calculate first conditional expectation                                                     
EV[0] = nested_mc_AC(S0, vol, r, q, 0, ts, 1000000,0)      

for i in range(1, len(ts)-1):
    Z = (i*pathsA[i]+(12-i)*pathsS[i])/12
    callZ = blackscholes_price(K, ts[-1]-ts[i], Z, vol_, callput='call')        
    cont_vals = betas_AC[i, 0]+betas_AC[i, 1]*callZ
    exer_vals =  np.maximum(pathsA[i] - K, 0)
    ind = cont_vals <= exer_vals
    for j in np.nonzero(ind)[0]:
        V[i, j] = exer_vals[j]*np.exp(-r*ts[i])                    
        EV[i,j] = nested_mc_AC(pathsS[i, j], vol, r, q, i, ts, nnested, pathsA[i,j]) 
    for j in np.nonzero(~ind)[0]:
        V[i,j] = nested_mc_AC(pathsS[i, j], vol, r, q, i, ts, nnested, pathsA[i,j])  
        EV[i, j] = V[i, j]   

#calculate final value of estimated snell envelope
V[-1] = np.maximum(pathsA[-1] - K, 0)*np.exp(-r*ts[-1])       

#calculate the martingale
hedges = np.zeros(pathsS.shape, dtype=np.float)
hedges[1:] = np.cumsum(V[1:]-EV[:-1], axis=0)               

#calculate discounted payoff less the martingale, find the max on each path, then average
LS_Value = np.mean(np.amax(((np.maximum(pathsA[1:] - K, 0)*np.exp(-r*ts[1:, np.newaxis]))-hedges[1:]), axis=0))

print("Lower Bound LS Value = {}".format(EV[0][0]))
print("Upper Bound LS Value = {}".format(LS_Value))

Lower Bound LS Value = 5.310781521752294
Upper Bound LS Value = 5.404012461040336


#### Question 4b.)

In [14]:
# Longstaff-Schwartz
S0, K, vol, r, q = 100, 100, 0.2, 0, 0
ts = np.linspace(0, 1, 13)
npaths = 100000
pathsS = blackscholes_mc(S0, vol, r, q, ts=ts, npaths=npaths)                                  # stock prices
pathsA = np.vstack((pathsS[0], np.cumsum(pathsS[1:], axis=0)/np.arange(1, 13)[:, np.newaxis])) # running averages
vol_ = 0.1
payoff = np.maximum(pathsA[-1]-K, 0)
betas_AC = np.full((len(ts), 6), np.nan, dtype=np.float)
for i in range(len(ts)-2, 0, -1):
    discount = np.exp(-r*(ts[i+1]-ts[i]))
    payoff = payoff*discount
    A = np.vstack((np.ones_like(pathsA[i]), pathsA[i], pathsS[i], pathsA[i] * pathsA[i], pathsS[i]*pathsS[i], pathsA[i]*pathsS[i])).T
    betas_AC[i] = np.linalg.lstsq(A, payoff, rcond=None)[0]           # regression to estimate continuation values
    contval = betas_AC[i, 0]+betas_AC[i, 1]*pathsA[i] +betas_AC[i, 2]* pathsS[i] +betas_AC[i, 3]* pathsA[i] * pathsA[i] + betas_AC[i, 4] * pathsS[i]*pathsS[i] + betas_AC[i, 5] * pathsA[i]*pathsS[i]
    exerval = np.maximum(pathsA[i]-K, 0)
    ind = exerval > contval                               # identify the paths where we should exercise
    payoff[ind] = exerval[ind]
#betas_AC

In [15]:
S0, K, vol, r, q, vol_ = 100, 100, 0.2, 0, 0, 0.1
ts = np.linspace(0, 1, 13)
nnested = 500 

# nested simulation from time t_i when stock price is S
def nested_mc_AC(S, vol, r, q, i, ts, nnested, Av):
    nested_paths = np.full(nnested, S, dtype=np.float)
    nested_Av = np.full(nnested, Av, dtype=np.float)
    tot_payoff = 0
    for j in range(i+1, len(ts)):
        dt = ts[j] - ts[j-1]
        dW = np.random.randn(len(nested_paths))*np.sqrt(dt)        # Brownian increment
        nested_paths = nested_paths*np.exp((r-q)*dt)*np.exp(-0.5*vol**2*dt + vol*dW)
        
        nested_Av = (nested_paths + (nested_Av * (j-1))) / (j)
        
        exer_vals =  np.maximum(nested_Av - K, 0)
        if j < len(ts)-1:
                  
            cont_vals = betas_AC[j, 0]+betas_AC[j, 1]*nested_Av +betas_AC[j, 2]* nested_paths +betas_AC[j, 3]* nested_Av * nested_Av + betas_AC[j, 4] * nested_paths *nested_paths + betas_AC[j, 5] * nested_Av*nested_paths 

            ind = cont_vals <= exer_vals
            tot_payoff += np.sum(exer_vals[ind])*np.exp(-r*ts[j])
            nested_paths = nested_paths[~ind]
            nested_Av = nested_Av[~ind]
            if len(nested_paths) == 0:                             
                break
        else:
            tot_payoff +=  np.sum(exer_vals)*np.exp(-r*ts[j])
    return tot_payoff/nnested                                      # taking average of paths

#generate 100000 independent paths
pathsS = blackscholes_mc(S0, vol, r, q, ts=ts, npaths=1000)                             
pathsA = np.vstack((pathsS[0], np.cumsum(pathsS[1:], axis=0)/np.arange(1, 13)[:, np.newaxis]))

#Andersen-Broadie algorithm similar to demonstration                                                    
V = np.full(pathsS.shape, np.nan, dtype=np.float)                   
EV = np.full(pathsS.shape, np.nan, dtype=np.float)                                                    

#calculate first conditional expectation                                                     
EV[0] = nested_mc_AC(S0, vol, r, q, 0, ts, 100000,0)      

for i in range(1, len(ts)-1):     
    cont_vals =betas_AC[i, 0]+betas_AC[i, 1]*pathsA[i] +betas_AC[i, 2]* pathsS[i] +betas_AC[i, 3]* pathsA[i] * pathsA[i] + betas_AC[i, 4] * pathsS[i]*pathsS[i] + betas_AC[i, 5] * pathsA[i]*pathsS[i]
    exer_vals =  np.maximum(pathsA[i] - K, 0)
    ind = cont_vals <= exer_vals
    for j in np.nonzero(ind)[0]:
        V[i, j] = exer_vals[j]*np.exp(-r*ts[i])                    
        EV[i,j] = nested_mc_AC(pathsS[i, j], vol, r, q, i, ts, nnested, pathsA[i,j]) 
    for j in np.nonzero(~ind)[0]:
        V[i,j] = nested_mc_AC(pathsS[i, j], vol, r, q, i, ts, nnested, pathsA[i,j])  
        EV[i, j] = V[i, j]   

#calculate final value of estimated snell envelope
V[-1] = np.maximum(pathsA[-1] - K, 0)*np.exp(-r*ts[-1])       

#calculate the martingale
hedges = np.zeros(pathsS.shape, dtype=np.float)
hedges[1:] = np.cumsum(V[1:]-EV[:-1], axis=0)               

#calculate discounted payoff less the martingale, find the max on each path, then average
LS_value = np.mean(np.amax(((np.maximum(pathsA[1:] - K, 0)*np.exp(-r*ts[1:, np.newaxis]))-hedges[1:]), axis=0))

print("Lower Bound LS Value = {}".format(EV[0][0]))
print("Upper Bound LS Value = {}".format(LS_value))

Lower Bound LS Value = 5.074362545769435
Upper Bound LS Value = 5.467038273138654


#### Question 4c.)

The duality gap in 4a.) using the BS functions from Homework 1 was relatively small, and usually around .08 USD. This indicates that the dual method and the primal methods are both accurate enough that they are converging towards the 'true' option price, rather than one presenting a very high upper bound and the other presenting a very low lower bound. 

The duality gap in 4b.) using the various products and sums of $A_{t_n}$ and $S_{t_n}$ had a much larger duality gap. This was mostly stemming from the lower bound estimate, as the upper bound estimate was similar to the low-biased estimate from Homework 1 and the upper bound estimate from 4a.) The gap was usually around .4 USD and therefore the basis functions aren't accurate enough to perform useful estimates of the conditional expectations in the primal method. 

I thought that it made sense for the BS functions to perform better than the products and sums of $A_{t_n}$ and $S_{t_n}$ since they are able to perform crude 'approximations' to American option prices. So when they are used in regression their outputs are always sensible, but when regression is used with just a linear combination of variables, the meaning of the variables and therefore the estimates are not always in line with the option pricing framework. 